# Integration through Stan

In [81]:
import nest_asyncio
nest_asyncio.apply()

import numpy as np
#from cmdstanpy import cmdstan_path, CmdStanModel
import stan
import matplotlib.pyplot as plt
import os
import tqdm

from mpmath import mp
mp.dps = 25; mp.pretty = True

import sys
sys.path.append('../scripts')

from parameter_estimation import BivariateBeta

First, let's analyze the `integrate_1d` function from Stan. We defined four different variations: 

1. The integrand is $u^{\alpha_1-1}(x-u)^{\alpha_2-1}(y-u)^{\alpha_3-1}(1-x-y+u)^{\alpha_4-1}$
2. The integrand is $\exp\{(\alpha_1-1)\cdot\log(u) + (\alpha_2-1)\cdot\log(x-u) + (\alpha_3-1)\cdot\log(y-u) + (\alpha_4-1)\cdot\log(1-x-y+u)\}$
3. The integrand is $u^{\alpha_1-1}(x-u)^{\alpha_2-1}(y-u)^{\alpha_3-1}(1-x-y+u)^{\alpha_4-1}$, but using the high precision parameter `xc` which is the distance between $u$ and the closest limit. For instance, if $u$ is close to $x < y$, we have $xc = x-u$, with more precision.
4. The integrand is$\exp\{(\alpha_1-1)\cdot\log(u) + (\alpha_2-1)\cdot\log(x-u) + (\alpha_3-1)\cdot\log(y-u) + (\alpha_4-1)\cdot\log(1-x-y+u)\}$, but using the high precision parameter `xc` which is the distance between $u$ and the closest limit.

In [2]:
filename = '../scripts/bivariate-beta-density.stan'
with open(filename) as f:
    model = f.read()

In [3]:
alpha = np.random.gamma(1,1,size=4)
n = 1000
XY = np.random.random(size=(n,2))
data = {'n': n, 'xy': XY, 'tolerance': 1e-10, 'alpha': alpha, 'integrand': 1}

In [5]:
data['integrand']=1
bivbeta_density = stan.build(model, data)
fit = bivbeta_density.fixed_param(num_chains=1, num_samples=1)

Building: found in cache, done.
Messages from stanc:
Warning in '/tmp/httpstan_vanpcngh/model_w4376tn3.stan', line 101, column 31: The
    variable integrand4 may not have been assigned a value before its use.
Warning in '/tmp/httpstan_vanpcngh/model_w4376tn3.stan', line 99, column 31: The
    variable integrand3 may not have been assigned a value before its use.
Warning in '/tmp/httpstan_vanpcngh/model_w4376tn3.stan', line 97, column 31: The
    variable integrand2 may not have been assigned a value before its use.
Warning in '/tmp/httpstan_vanpcngh/model_w4376tn3.stan', line 95, column 31: The
    variable integrand1 may not have been assigned a value before its use.
Sampling:   0%
Sampling: 100% (1/1)
Sampling: 100% (1/1), done.
Messages received during sampling:
  Exception: Exception: Error in function tanh_sinh<double>::integrate: The tanh_sinh quadrature evaluated your function at a singular point and got inf. Please narrow the bounds of integration or check your function for si

In [6]:
data['integrand']=2
bivbeta_density = stan.build(model, data)
fit = bivbeta_density.fixed_param(num_chains=1, num_samples=1)
a2 = fit['log_density']

Building: found in cache, done.
Messages from stanc:
Warning in '/tmp/httpstan_vanpcngh/model_w4376tn3.stan', line 101, column 31: The
    variable integrand4 may not have been assigned a value before its use.
Warning in '/tmp/httpstan_vanpcngh/model_w4376tn3.stan', line 99, column 31: The
    variable integrand3 may not have been assigned a value before its use.
Warning in '/tmp/httpstan_vanpcngh/model_w4376tn3.stan', line 97, column 31: The
    variable integrand2 may not have been assigned a value before its use.
Warning in '/tmp/httpstan_vanpcngh/model_w4376tn3.stan', line 95, column 31: The
    variable integrand1 may not have been assigned a value before its use.
Sampling:   0%
Sampling: 100% (1/1)
Sampling: 100% (1/1), done.
Messages received during sampling:
  Exception: Exception: Error in function tanh_sinh<double>::integrate: The tanh_sinh quadrature evaluated your function at a singular point and got inf. Please narrow the bounds of integration or check your function for si

In [7]:
data['integrand']=3
bivbeta_density = stan.build(model, data)
fit = bivbeta_density.fixed_param(num_chains=1, num_samples=1)
a3 = fit['log_density']

Building: found in cache, done.
Messages from stanc:
Warning in '/tmp/httpstan_vanpcngh/model_w4376tn3.stan', line 101, column 31: The
    variable integrand4 may not have been assigned a value before its use.
Warning in '/tmp/httpstan_vanpcngh/model_w4376tn3.stan', line 99, column 31: The
    variable integrand3 may not have been assigned a value before its use.
Warning in '/tmp/httpstan_vanpcngh/model_w4376tn3.stan', line 97, column 31: The
    variable integrand2 may not have been assigned a value before its use.
Warning in '/tmp/httpstan_vanpcngh/model_w4376tn3.stan', line 95, column 31: The
    variable integrand1 may not have been assigned a value before its use.
Sampling:   0%
Sampling: 100% (1/1)
Sampling: 100% (1/1), done.


In [13]:
data['integrand']=4
bivbeta_density = stan.build(model, data)
fit = bivbeta_density.fixed_param(num_chains=1, num_samples=1)
quantities_stan = fit['log_density']

Building: found in cache, done.
Messages from stanc:
Warning in '/tmp/httpstan_vanpcngh/model_w4376tn3.stan', line 101, column 31: The
    variable integrand4 may not have been assigned a value before its use.
Warning in '/tmp/httpstan_vanpcngh/model_w4376tn3.stan', line 99, column 31: The
    variable integrand3 may not have been assigned a value before its use.
Warning in '/tmp/httpstan_vanpcngh/model_w4376tn3.stan', line 97, column 31: The
    variable integrand2 may not have been assigned a value before its use.
Warning in '/tmp/httpstan_vanpcngh/model_w4376tn3.stan', line 95, column 31: The
    variable integrand1 may not have been assigned a value before its use.
Sampling:   0%
Sampling: 100% (1/1)
Sampling: 100% (1/1), done.


We want to compare with the Appell F1 implementation in Python. Both are numerical approximations to the true value, however the implementation below is more robust. 

In [11]:
biv_beta_object = BivariateBeta(alpha=alpha)
quantities_scipy = np.array([np.log(float(biv_beta_object.pdf_appell(XY[i,0], XY[i,1]))) for i in range(n)]).reshape(-1,1)

The maximum percentage error is

In [15]:
np.max(np.abs(quantities_scipy - quantities_stan)/quantities_scipy)

1.3830862310777245e-14

## Stan model

In [16]:
filename = '../scripts/bivariate-beta-model.stan'

Generating data.

In [61]:
alpha = np.array([1,1,1,1])
n = 50
U = np.random.dirichlet(alpha, size=n)
X = U[:,0] + U[:,1]
Y = U[:,0] + U[:,2]
XY = np.column_stack([X,Y])

a = np.array([1,1,1,1])
b = np.array([1,1,1,1])

data = {'n': n, 'xy': XY, 'a': a, 'b': b, 'tolerance': 1e-8}

In [63]:
with open(filename) as f:
    model = f.read()

bivbeta_model = stan.build(model, data)

Building: found in cache, done.
Messages from stanc:
Warning in '/tmp/httpstan_xz6pxvrk/model_z5odsxxc.stan', line 41, column 31: The
    variable integrand may not have been assigned a value before its use.


Sampling

In [64]:
model_fit = bivbeta_model.sample(num_chains=4, num_samples=1000)

Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  11% (900/8000)
Sampling:  12% (1000/8000)
Sampling:  14% (1100/8000)
Sampling:  15% (1200/8000)
Sampling:  16% (1300/8000)
Sampling:  18% (1400/8000)
Sampling:  19% (1500/8000)
Sampling:  20% (1600/8000)
Sampling:  21% (1700/8000)
Sampling:  22% (1800/8000)
Sampling:  24% (1900/8000)
Sampling:  25% (2000/8000)
Sampling:  26% (2100/8000)
Sampling:  28% (2200/8000)
Sampling:  29% (2300/8000)
Sampling:  30% (2400/8000)
Sampling:  31% (2500/8000)
Sampling:  32% (2600/8000)
Sampling:  34% (2700/8000)
Sampling:  35% (2800/8000)
Sampling:  36% (2900/8000)
Sampling:  38% (3000/8000)
Sampling:  39% (3100/8000)
Sampling:  40% (3200/8000)
Sampling:  41% (3300/8000)
Samplin

In [67]:
print(np.quantile(model_fit['alpha'], axis=1, q=0.5))
print(model_fit['alpha'].mean(axis=1))

[0.80598714 1.03843482 0.99042469 0.91154168]
[0.81123072 1.04270837 0.99755572 0.91654255]


In [78]:
model_fit.values()

ValuesView(<stan.Fit>
Parameters:
    alpha: (4,)
Draws: 4000)